# State and Lifecycle

Let's first look at our clock example from before:

In [ ]:
const root = ReactDOM.createRoot(document.getElementById('root'));
  
function tick() {
  const element = (
    <div>
      <h1>Hello, world!</h1>
      <h2>It is {new Date().toLocaleTimeString()}.</h2>
    </div>
  );
  root.render(element);
}

setInterval(tick, 1000);

Let's make a `Clock` component that will have its OWN timer and update ITSELF every second.  
We can start by encapsulating the `Clock`:

In [ ]:
const root = ReactDOM.createRoot(document.getElementById('root'));

function Clock(props) {
  return (
    <div>
      <h1>Hello, world!</h1>
      <h2>It is {props.date.toLocaleTimeString()}.</h2>
    </div>
  );
}

function tick() {
  root.render(<Clock date={new Date()} />);
}

setInterval(tick, 1000);

No problem is that the `Clock` does not have its OWN timer, and the update code needs to be part of the implementation of `Clock`.  
This is how we ideally want to render itself:

In [ ]:
root.render(<Clock />);

But what is the problem with this?:

In [ ]:
const root = ReactDOM.createRoot(document.getElementById('root'));

function Clock() {
  return (
    <div>
      <h1>Hello, world!</h1>
      <h2>It is {new Date().toLocaleTimeString()}.</h2>
    </div>
  );
}

function tick() {
  root.render(<Clock />);
}

setInterval(tick, 1000);

The problem, is that the `Clock` does not have its own timer.

To achieve this, we need to add "state" to the `Clock` component. State is similar to props, but is private and controlled by the component.

## Adding Local State to a Class

In order to use "state", we need to use the ES6 class component rather than a function component. Here is the same code, but using the ES6 class:

In [ ]:
const root = ReactDOM.createRoot(document.getElementById('root'));

class Clock extends React.Component {
  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.props.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

function tick() {
  root.render(<Clock date={new Date()} />);
}

setInterval(tick, 1000);

The `render` method of `Clock` will be called each time an update happens, so when `root.render()` occurs. But since `<Clock />` is in the same DOM node, only one instance of `Clock` is used.

Here we will then move `date` from props to state in three steps:

In [ ]:
class Clock extends React.Component {
  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.state.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

Take a note of what we changed here.

In [ ]:
class Clock extends React.Component {
  constructor(props) {
    super(props);
    this.state = {date: new Date()};
  }

  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.state.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

Make sure you fully understand everything that is going on in this step. Like when is `constructor` being called, what `super(props)` does, and what `this.state = ...` actually means.

#### NOTE: Class components should ALWAYS call the base constructor with `props`.

Remove the `date` prop from the `<Clock />` element, and we are done:

In [ ]:
class Clock extends React.Component {
  constructor(props) {
    super(props);
    this.state = {date: new Date()};
  }

  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.state.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

const root = ReactDOM.createRoot(document.getElementById('root'));
root.render(<Clock />);

But, the `Clock` does NOT update itself.

## Adding Lifecycle Methods to a Class

We want to set up a timer to `Clock` when it is rendered for the first time. This is called "mounting" in React.<br>
Next we want to clear the timer when the `Clock` is removed. This is called "unmounting" in React.

Here are the special methods to run when a component mounts and unmounts:

In [ ]:
class Clock extends React.Component {
  constructor(props) {
    super(props);
    this.state = {date: new Date()};
  }

  componentDidMount() {
  }

  componentWillUnmount() {
  }

  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.state.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

These two methods are **Lifecycle Methods**.

`componentDidMount()` runs when component is rendered to the DOM. So we will set the timer here:

In [ ]:
componentDidMount() {
    this.timerID = setInterval(  () => this.tick()  , 1000);
}

Note how `this.timerID` is created here. We will use this later when we clear the timer we made here.

Note `this.props` is already set up by React itself, and its value is the `props` we passed in. `this.state` is assigned when we made it in the constructor.

Here we unmount the timer:

In [ ]:
componentWillUnmount() {
    clearInterval(this.timerID);
}

Now we need to implement `tick()` that will run every second:

In [ ]:
tick() {
    this.setState( {date: new Date()} );
}

Note how similar this looks like to when we first made `this.state = {data: new Date()};` in the constructor.

Here is the final code:

In [ ]:
class Clock extends React.Component {
  constructor(props) {
    super(props);
    this.state = {date: new Date()};
  }

  componentDidMount() {
    this.timerID = setInterval(
      () => this.tick(),
      1000
    );
  }

  componentWillUnmount() {
    clearInterval(this.timerID);
  }

  tick() {
    this.setState({date: new Date()});
  }

  render() {
    return (
      <div>
        <h1>Hello, world!</h1>
        <h2>It is {this.state.date.toLocaleTimeString()}.</h2>
      </div>
    );
  }
}

const root = ReactDOM.createRoot(document.getElementById('root'));
root.render(<Clock />);

### Note, that you cannot modify state directly!!

For example, this won't work:

In [ ]:
tick(){
    this.state = {date: new Date()};
}

Nor this (because this.state is an object):

In [ ]:
tick(){
    this.state.date = new Data();
}

### The ONLY place you can assign `this.state` is the constructor!!

## State Updates May Be Asynchronous

React may batch multiple `setState()` calls into a single update for performance. Because `this.props` and `this.state` may be updated asynchronously (one may be updated before the other has been updated), you should not rely on the values for calculating the next state:

In [ ]:
this.setState(  {counter: this.state.counter + this.props.increment}  );

In this example, the OLD `increment` could still exist. To fix this, you need to use a second form of `setState()` that accepts a function rather than an object. It will get the previous state as the first argument, and the props at the time the UPDATE IS APPLIED as the second argument:

In [2]:
this.setState((state,props) => (  {counter: state.counter + props.increment}  ));

Error: Unexpected token ';'

Above, we used the arrow function, but we can also use regular functions:

In [ ]:
this.setState(function(state,props) {
    return {counter: state.counter + props.increment}
});

## State Updates are Merged

When calling `setState()`, React merges the object you provide into the current state. For example:

In [ ]:
 constructor(props) {
    super(props);
    this.state = {
      posts: [],
      comments: []
    };
  }

In [ ]:
componentDidMount() {
    fetchPosts().then(response => {
      this.setState({
        posts: response.posts
      });
    });

    fetchComments().then(response => {
      this.setState({
        comments: response.comments
      });
    });
  }

Here, we independently update the variables using different `this.setState()` calls. In the first one, `this.setState({posts})` will leave `this.state.comments` intact, but replace the current `this.state.posts`.